In [37]:
# Importaciones e instalaciones necesarias
from selenium import webdriver
import random 
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager # pip install webdriver-manager

# Para trabajar con mongodb
!pip install pymongo
from pymongo import MongoClient
client = MongoClient('localhost')
db = client['pruebaTusDatos']
col = db['procesos']
detalles = db['detalles']

In [38]:
opts = Options()
opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36")

# Conexión con la página web a "scrapear"
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options=opts)
driver.get('https://procesosjudiciales.funcionjudicial.gob.ec/busqueda-filtros')

# Se establece el tipo de proceso a realizar
type_process = int(input("Desea ingresar código para: \n"+
                   "1. Demandante\n"+
                   "2. Demandado\n"))
# Código del demandante o demandado
# Demandante1 = 0968599020001
# Demandante2 = 0992339411001

# Demandado1 = 1791251237001
# Demandado2 = 0968599020001
codigo = input("Ingrese el número de identificación: ")



Desea ingresar código para: 
1. Demandante
2. Demandado
1
Ingrese el número de identificación: 0968599020001


In [39]:
# Me doy cuenta que la pagina carga el formulario dinamicamente luego de que la carga incial ha sido completada
# Por eso tengo que esperar que aparezca 
if type_process == 1:
    input_user = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//input[@formcontrolname = 'cedulaActor']")))
    input_user.send_keys(codigo)
elif type_process == 2:
    input_user = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//input[@formcontrolname = 'cedulaDemandado']")))
    input_user.send_keys(codigo)


# Click al boton buscar
boton_buscar = driver.find_element(By.XPATH,"//button[@type = 'submit']")
boton_buscar.click()


# Trabajo con DETALLES

### Función para guardar los detalles en la base de datos

In [40]:
def enviar_detalles(numero_proceso, listado_detalles):
    for detalle in listado_detalles:
        #fecha_ingreso = detalle.find_element('xpath','.//')
        titulo_detalle = detalle.find_element('xpath','.//span[contains(@class, \"title\")]').text
        
        detalles.insert_one({
            'titulo_detalle': titulo_detalle,
            'numero_proceso': numero_proceso
        })

### Función para buscar los detalles

In [41]:
# Obtención del detalle
# Obtengo el listado de procesos esperando antes 10 segundos para dar tiempo de que se carguen por completo
# listado_procesos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class = 'causa-individual ng-star-inserted']")))

#print(listado_procesos[0].text)
def obtencion_detalle(proceso):
    try:
        # Se espera unos segundos antes de dar click al boton de detalles
        sleep(random.uniform(8.0,10.0))

        numero_proceso = proceso.find_element(By.XPATH,'.//div[@class="numero-proceso"]').text
        boton_proceso = proceso.find_element(By.XPATH,'.//a[@href="/movimientos"]')
        boton_proceso.click()

        # Se espera unos segundos antes de dar click al boton de detalles
        sleep(random.uniform(8.0,10.0))
        boton_detalle_proceso = driver.find_element(By.XPATH,'.//a[@href="/actuaciones"]')
        boton_detalle_proceso.click()
        # Se extraen todos los detalles según la fecha de ingreso
        #listado_detalles = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[contains(@class = 'cabecera-tabla')]")))
        listado_detalles = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//span[contains(@class, \"mat-content\")]")))

        # Se envian los detalles a la base de datos
        enviar_detalles(numero_proceso,listado_detalles)

        sleep(random.uniform(3.0,5.0))
        driver.back()
        driver.back()
        
    except StaleElementReferenceException:
        # Reintentar localizar y obtener detalles del proceso
        sleep(2)  # Puede ajustarse según necesidades
        obtencion_detalle(proceso)

#obtencion_detalle(listado_procesos[0])



# Trabajo con PROCESO

### Función para enviar un proceso a la base de datos

In [42]:
def enviar_proceso(listado_proceso):
    for proceso in listado_proceso:
        id_proceso = proceso.find_element('xpath','.//div[@class="id"]').text
        fecha = proceso.find_element('xpath','.//div[@class="fecha"]').text
        numero_proceso = proceso.find_element('xpath','.//div[@class="numero-proceso"]').text
        accion_infraccion = proceso.find_element('xpath','.//div[@class="accion-infraccion"]').text
        
        # Se envia la información obtenida a la base de datos
        col.insert_one({
            'id':id_proceso,
            'fecha': fecha,
            'numero_proceso': numero_proceso,
            'accion_infraccion': accion_infraccion
        })

In [12]:
# Se obtiene el número limite para realizar los cambios de página
page_label = driver.find_element(By.XPATH, "//div[@class = 'mat-mdc-paginator-range-label']")
text = page_label.text

# Se utiliza la funcion split para obtener un array por palabra del texto obtenido y luego se obtiene el ultimo registro
parts = text.split()
limit = int(parts[len(parts) - 1])

In [35]:
# Obtengo el listado de procesos esperando antes 10 segundos para dar tiempo de que se carguen por completo
listado_procesos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class = 'causa-individual ng-star-inserted']")))
# Se envia la lista de procesos encontrados por pantalla a la base de datos
enviar_proceso(listado_procesos)

# Ciclo para recorrer el listado de procesos y poder obtener y enviar los detalles de cada uno
for proceso in listado_procesos:
    try:
        obtencion_detalle(proceso)
    except StaleElementReferenceException:
        # Reubicar en caso de excepción
        listado_procesos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class='causa-individual ng-star-inserted']")))
        obtencion_detalle(proceso)

# Se obtiene el boton de paso
boton_paso = driver.find_element(By.XPATH,"//button[@aria-label='Página siguiente']")
                                                           
# Se recorren todas las vistas de los registros
for i in range(4):
    try:
        # click al boton de paso
        boton_paso.click()
        # Espero a que se cargue la nueva página
        sleep(random.uniform(8.0,10.0))
        # Obtengo el listado de procesos esperando antes 10 segundos para dar tiempo de que se carguen por completo
        listado_procesos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class = 'causa-individual ng-star-inserted']")))
        # Se envia la lista de procesos encontrados por pantalla a la base de datos
        enviar_proceso(listado_procesos)
        
        for proceso in listado_procesos:
            try:
                obtencion_detalle(proceso)
            except StaleElementReferenceException:
                # Reubicar en caso de excepción
                listado_procesos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class='causa-individual ng-star-inserted']")))
                obtencion_detalle(proceso)
            
        # Espero a que se cargue la nueva página
        sleep(random.uniform(8.0,10.0))
        # Se obtiene el boton de paso
        boton_paso = driver.find_element(By.XPATH,"//button[@aria-label='Página siguiente']")
    except StaleElementReferenceException:
        # Reubicar el botón de paso en caso de excepción
        boton_paso = driver.find_element(By.XPATH,"//button[@aria-label='Página siguiente']")
    except Exception as e:
        print(e)
        break

NameError: name 'StaleElementReferenceException' is not defined

In [43]:
# Obtengo el listado de procesos esperando antes 10 segundos para dar tiempo de que se carguen por completo
listado_procesos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class = 'causa-individual ng-star-inserted']")))
# Se envia la lista de procesos encontrados por pantalla a la base de datos
#enviar_proceso(listado_procesos)
array_numeros_procesos = []
# Ciclo para recorrer el listado de procesos y poder obtener y enviar los detalles de cada uno
for n in range(9):
    # Se obtienen y envian los detalles a la base de datos
    array_numeros_procesos.append(listado_procesos[n].find_element('xpath','.//div[@class="numero-proceso"]').text)

for numero in array_numeros_procesos:
    for proceso in listado_procesos:
    # Se envia la lista de procesos encontrados por pantalla a la base de datos
        sleep(random.uniform(5.0,7.0))
        if numero == proceso.find_element('xpath','.//div[@class="numero-proceso"]').text :
            obtencion_detalle(proceso)
        listado_procesos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class = 'causa-individual ng-star-inserted']")))
        
    

StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=126.0.6478.63); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00AC0ED3+27667]
	(No symbol) [0x00A56BD4]
	(No symbol) [0x00951AFF]
	(No symbol) [0x00956BEF]
	(No symbol) [0x00958878]
	(No symbol) [0x009588F0]
	(No symbol) [0x00992847]
	(No symbol) [0x00992CDB]
	(No symbol) [0x00988C91]
	(No symbol) [0x009B3974]
	(No symbol) [0x00988BA5]
	(No symbol) [0x009B3BC4]
	(No symbol) [0x009CCAC7]
	(No symbol) [0x009B36C6]
	(No symbol) [0x009874D1]
	(No symbol) [0x0098804D]
	GetHandleVerifier [0x00D73D43+2857603]
	GetHandleVerifier [0x00DC7EAD+3202029]
	GetHandleVerifier [0x00B446D4+566292]
	GetHandleVerifier [0x00B4B8CC+595468]
	(No symbol) [0x00A5F9C4]
	(No symbol) [0x00A5C6D8]
	(No symbol) [0x00A5C877]
	(No symbol) [0x00A4

In [18]:
lista1 = [1,2,3]
for item in lista1:
    if item == 2:
        print("El 2 existe")

El 2 existe


In [25]:
print(array_numeros_procesos)

['13284202419612', '12282202406107', '08332202400176', '13284202406765', '13284202406764', '13284202406763', '17811202302644', '09802202301613', '09332202318810']


In [26]:
array_numeros_procesos[0]

'13284202419612'

In [27]:
array_numeros_procesos[1]

'12282202406107'